In [23]:
import os
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel,InferenceClientModel, OpenAIServerModel
from langchain_community.document_loaders import WikipediaLoader
from smolagents import tool
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import helium

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--force-device-scale-factor=1")
chrome_options.add_argument("--window-size=1000,1350")
chrome_options.add_argument("--disable-pdf-viewer")
chrome_options.add_argument("--window-position=0,0")

# Initialize the browser
driver = helium.start_chrome(headless=False, options=chrome_options)


haiku = 'anthropic/claude-3-5-haiku-20241022'
sonnet = 'anthropic/claude-sonnet-4-20250514'

# claude model
model = LiteLLMModel(
    model_id = haiku,
    api_key=os.environ['CLAUDE_API']
)


In [24]:
# wiki search
@tool
def wiki_search(query: str) -> str:
    """Search Wikipedia for a query and return maximum 2 results.
    
    Args:
        query: The search query."""
    search_docs = WikipediaLoader(query=query, load_max_docs=2).load()
    formatted_search_docs = "\n\n---\n\n".join(
        [
            f'<Document source="{doc.metadata["source"]}" page="{doc.metadata.get("page", "")}"/>\n{doc.page_content}\n</Document>'
            for doc in search_docs
        ])
    return {"wiki_results": formatted_search_docs}

# duckduck
duckduckseach = DuckDuckGoSearchTool()

# selenium
# @tool
# def open_browser():
#     driver = helium.start_chrome(headless=False, options=chrome_options)

@tool
def search_item_ctrl_f(text: str, nth_result: int = 1) -> str:
    """
    Searches for text on the current page via Ctrl + F and jumps to the nth occurrence.
    Args:
        text: The text to search for
        nth_result: Which occurrence to jump to (default: 1)
    """
    elements = driver.find_elements(By.XPATH, f"//*[contains(text(), '{text}')]")
    if nth_result > len(elements):
        raise Exception(f"Match n°{nth_result} not found (only {len(elements)} matches found)")
    result = f"Found {len(elements)} matches for '{text}'."
    elem = elements[nth_result - 1]
    driver.execute_script("arguments[0].scrollIntoView(true);", elem)
    result += f"Focused on element {nth_result} of {len(elements)}"
    return result


@tool
def go_back() -> None:
    """Goes back to previous page."""
    driver.back()


@tool
def close_popups() -> str:
    """
    Closes any visible modal or pop-up on the page. Use this to dismiss pop-up windows! This does not work on cookie consent banners.
    """
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()


In [26]:
tools_list = [wiki_search, duckduckseach, close_popups, go_back, search_item_ctrl_f]

flight_agent = CodeAgent(
    model=model,
    tools = tools_list,
    additional_authorized_imports=['numpy', 'helium'],
    max_steps=2
)

In [ ]:
system_prompt = '''
You are a helpful assistant tasked with planning travels for user. You should look in google flight for flights corresponding to some constrains. 
You can navigate a browser using 
Your output should follow the template
Dates = [arrival to destination date - departure from destination date],
budget = [budget in euro],
link = [link to google flight corresponding to the flight]

'''
question = 'Find me a flight to barcelona from Paris between the 24 and 26 of june 2025 for a budget less than 300 euros.'
flight_agent.run(system_prompt + question)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are a helpful assistant tasked with planning travels for user. You should look in google flight for flights │
│ corresponding to some constrains.                                                                               │
│ Your output should follow the template                                                                          │
│ Dates = [dates\],                                                                                               │
│ budget = [budget\],                                                                                             │
│ link = [link to google flight corresponding to the flight\]                                                     │
│                                                                                                                 │
│ Find me a flight to barcelona from Paris between the 24 and 26 of june 2025 for a budget less than 300 euros.   │
│                                                                                                                 │
╰─ LiteLLMModel - anthropic/claude-3-5-haiku-20241022 ────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_query = "flights from Paris to Barcelona June 24-26 2025 under 300 euros google flights"                  
  flight_search_results = web_search(query=search_query)                                                           
  print(flight_search_results)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Google Flights - Find Cheap Flight Options & Track Prices](https://www.google.com/travel/flights)
Use Google Flights to explore cheap flights to anywhere. Search destinations and track prices to find and book your
next flight.

[$17 Flights from Paris (PARI) to Barcelona (BCN) | 
Skyscanner](https://www.skyscanner.com/routes/pari/bcn/paris-to-barcelona.html)
It looks like Paris to Barcelona flights are currently cheapest in August. Jun. from $32. Jul. from $37. Aug. from 
$17. Sep. from $17. Oct. from $32. Nov. from $32. Dec. from $45. Jan. from $45. Feb. from $46. Mar. from $46. Apr. 
from $64. May. ... There are 713 flights per week flying from Paris to Barcelona, as of June 2025.

[$47 CHEAP FLIGHTS from Paris to Barcelona (PAR - BCN) - 
KAYAK](https://www.kayak.com/flight-routes/Paris-PAR/Barcelona-El-Prat-BCN)
Find flights to Barcelona from $47. Fly from Paris on Vueling, Transavia France and more. ... there are between 23 
and 26 nonstop flights that take off from Paris and land in Barcelona-El Prat Airport, with an average flight time 
of 1h 44m. The most common departure time is 9:00 am and most flights take off in the morning. ... 24/7. Do Paris 
...

[$47+ Flights from Paris to Barcelona-El Prat Airport | 
Cheapflights](https://www.cheapflights.com/flights-to-barcelona-intl/paris/)
Customize your flight search to find flights from Paris to Barcelona-El Prat that fit within your specific budget. 
Explore options based on user searches and filter the results based on your desired price range. Find flights that 
match your budget for a cost-effective journey. Over the last 7 days, Cheapflights users made 2,558,603 searches.

[$47 Cheap Flights from Paris to Barcelona in 2025 | momondo](https://www.momondo.com/flights/paris/barcelona)
Search and compare airfare from 1000+ airlines and travel sites to get the cheapest flights from Paris to Barcelona
with momondo. ... Sep 22 - Wed, Sep 24. 10:35 am - 12:30 pm CDG-BCN. 1h 55m nonstop. 7:25 am - 9:25 am BCN-CDG. 2h 
00m nonstop. $85 Vueling. Find Deal. ... There are 4 operators available for flights from Paris to Barcelona as of 
...

[Cheap flights from Paris to Barcelona from $42 - 
Kiwi.com](https://www.kiwi.com/us/cheap-flights/paris-france/barcelona-spain/)
The distance between Paris and Barcelona is 908 km. The most popular airlines for this route are Vueling, Air 
France, Ryanair, Iberia Airlines, and Turkish Airlines. Paris and Barcelona have 400 direct flights per week. When 
you arrive at Barcelona, consider visiting Camp Nou, Sagrada Familia, Museu Picasso - Barcelona, and Teatre-Museu 
Dalí.

[Cheap Flights from Paris (PARI) to Barcelona 
(BCN)](https://www.skyscanner.net/routes/pari/bcn/paris-to-barcelona.html)
Compare cheap flights and find tickets from Paris (PARI) to Barcelona (BCN). Book directly with no added fees. ... 
As of June 2025, there are 820 flights per week flying from Paris to Barcelona. ... (about £7.95). A cheaper option
is the local bus, route 46, which stops at both terminals and takes just under an hour to reach the city centre ...

[Cheap Flights from Paris to Barcelona 2025 (PAR - 
Trip.com](https://www.trip.com/flights/paris-to-barcelona/airfares-par-bcn/)
Book one-way or round-trip flights from Paris to Barcelona operated by popular airlines with latest luggage & 
cancellation policy information! ... Tap "Date" on flight search bar to scan through the calender for Paris to 
Barcelona airfare over next 6 months in 2025. If you would like to fly in next 3 months, it seems the cheapest date
is on Tue ...

[Direct flights from Paris to Barcelona - 
FlightConnections](https://www.flightconnections.com/flights-from-cdg-to-bcn)
All direct (non-stop) flights from Paris to Barcelona. See airlines, aircrafts, flight time and distance, flight 
schedules and timetables. ... There are 71 flights per week flying from Paris to Barcelona (as of June 2025). ... 
Aircraft types that fly from Paris to Barcelona: Airbus A

[Step 1: Duration 4.14 seconds| Input tokens: 2,647 | Output tokens: 92]

...

Execution logs:
Dates: ['2025-06-24', '2025-06-25', '2025-06-26']
Budget: 300 euros
Link: 
https://www.google.com/travel/flights?q=Flights%20from%20Paris%20to%20Barcelona%20from%202025-06-24%20to%202025-06-
26%20under%20300%20EUR

Out: None

[Step 2: Duration 5.52 seconds| Input tokens: 6,807 | Output tokens: 295]

Reached max steps.

[Step 3: Duration 4.80 seconds| Input tokens: 9,082 | Output tokens: 492]

'Dates = ["2025-06-24", "2025-06-25", "2025-06-26"],\nbudget = 300 euros,\nlink = https://www.google.com/travel/flights?q=Flights%20from%20Paris%20to%20Barcelona%20from%202025-06-24%20to%202025-06-26%20under%20300%20EUR\n\nMultiple flight search sources indicate flights between Paris and Barcelona during this period are typically priced around $47-$60 (approximately 40-55 euros), which is well within the specified budget. Airlines like Vueling, Transavia France, and Air France operate this route with multiple daily flights. The flight time is approximately 1 hour 45 minutes, and there are around 70-80 flights per week between these cities in June 2025.'

Outputs of previous agents
---

In [ ]:
google_flight_link = 'https://www.google.com/travel/flights/booking?tfs=CBwQAhpKEgoyMDI1LTA2LTIxIiAKA0NERxIKMjAyNS0wNi0yMRoDQkNOKgJWWTIEODI0MWoMCAISCC9tLzA1cXRqcgwIAxIIL20vMDFmNjIaShIKMjAyNS0wNi0yNSIgCgNCQ04SCjIwMjUtMDYtMjUaA0NERyoCVlkyBDgyNDhqDAgDEggvbS8wMWY2MnIMCAISCC9tLzA1cXRqQAFIAXABggELCP___________wGYAQE&tfu=CmxDalJJYmtnNGFrOXpkVmRyUWtsQlRtcExlVkZDUnkwdExTMHRMUzB0TFhkbWFHVTBNVUZCUVVGQlIyaFBjRmR6UzBsWVdFbEJFZ1pXV1RneU5EZ2FDZ2ppU3hBQ0dnTkZWVkk0SEhESFZ3PT0SAggAIgA'

Manager agent
---

In [ ]:
# define all agents
# managed_agents = [flight_agent, airbnb_agent, activities_agent]
managed_agents = [flight_agent]
# final checks

# manager agent
manager_agent = CodeAgent(
    model= LiteLLMModel(model_id = sonnet, api_key=os.environ['CLAUDE_API']),
    tools=[],
    managed_agents=managed_agents,
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    # final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)


In [ ]:
manager_system_prompt = '''
You are a helful assistant whose mission is to plan travels for users. 
You can manage 3 agents: 
a flight agent that can find flights on google flights and outputs the dates, budget and url for booking,
an airbnb agent that can find houses on airbnb and outputs the dates budget and url for booking purposes
an activities agent that can propose activities and outputs a list of activities

Take the inputs of the three agents and combine the outputs into 3 differents travel plans with 3 budget tiers: low cost, medium cost and high cost
your output should follow the template:

'''

task = ''
manager_agent.run(manager_system_prompt + task)